In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import contextlib
import pandas as pd
import time
URL ="https://mt.metrc.com/log-in?ReturnUrl=%2findustry%2fD-100781-001%2fpackages"
PKG_URL = "https://mt.metrc.com/industry/D-100781-001/packages"

In [2]:

# Create Firefox options
options = Options()
options.headless = False

/var/folders/wx/nv6f5h9d043f4pfdzdjtt8j80000gr/T/ipykernel_47240/4080598429.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = False


In [3]:
driver  = webdriver.Firefox(executable_path='./geckodriver', options=options)

/var/folders/wx/nv6f5h9d043f4pfdzdjtt8j80000gr/T/ipykernel_47240/2202354667.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver  = webdriver.Firefox(executable_path='./geckodriver', options=options)


In [4]:
# Set the maximum wait time to 10 seconds
wait = WebDriverWait(driver, 4)

# Wait until the element with ID "myElement" is visible
# element = wait.until(EC.visibility_of_element_located((By.ID, "myElement")))


In [5]:
find = lambda path : driver.find_element(By.XPATH, path)
find_em = lambda path : driver.find_elements(By.XPATH, path)
click = lambda path : wait.until(EC.visibility_of_element_located((By.XPATH, path))).click()
enter = lambda path, text : driver.find_element(By.XPATH, path).send_keys(text)

In [6]:
# XPATHS
INACTIVE_PKG_TAB = '/html/body/div[3]/div[2]/div[2]/div/div/div[1]/ul/li[3]/span[2]'

ITEM_FILTER_3DOT = '/html/body/div[3]/div[2]/div[2]/div/div/div/div[3]/div/table/thead/tr/th[10]/a[1]/span'
ITEM_FILTER_OPTION = '/html/body/div[13]/div/ul/li[6]/span'
ITEM_FILTER_INPUT = '/html/body/div[13]/div/ul/li[6]/div/ul/li/div/form/div/input[1]'
ITEM_FILTER_BUTTON = '/html/body/div[13]/div/ul/li[6]/div/ul/li/div/form/div/div[2]/button[1]'

LOC_FILTER_3DOT = '/html/body/div[3]/div[2]/div[2]/div/div/div/div[3]/div/table/thead/tr/th[7]/a[1]/span'
LOC_FILTER_OPTION = '/html/body/div[14]/div/ul/li[6]/span'
LOC_FILTER_INPUT_1 = '/html/body/div[14]/div/ul/li[6]/div/ul/li/div/form/div/input[1]'
LOC_FILTER_INPUT_2 = '/html/body/div[14]/div/ul/li[6]/div/ul/li/div/form/div/input[2]'
LOC_FILTER_BUTTON = '/html/body/div[14]/div/ul/li[6]/div/ul/li/div/form/div/div[2]/button[1]'
LOC_FILTER_LOGIC_BUTTON = '/html/body/div[14]/div/ul/li[6]/div/ul/li/div/form/div/span[2]/span/span[1]'
LOC_FILTER_LOGIC_OR = '/html/body/div[14]/div/ul/li[6]/div/ul/div[2]/div/div[2]/ul/li[2]'

In [20]:
def login():
    with contextlib.suppress(Exception):
        username = driver.find_element(By.ID,'username')
        username.send_keys('102391')

        password = driver.find_element(By.ID, 'password')
        password.send_keys('7924Trill!')
        password.send_keys(Keys.ENTER)
        time.sleep(2)
    

In [8]:
def goto_inactive_pkg():
    driver.get(PKG_URL)
    time.sleep(1.5)
    click(INACTIVE_PKG_TAB)
    time.sleep(0.5)


In [9]:
def filter_item(item):
    click(ITEM_FILTER_3DOT)
    click(ITEM_FILTER_OPTION)
    enter(ITEM_FILTER_INPUT, item)
    click(ITEM_FILTER_BUTTON)
    

In [10]:
def filter_loc(item_type):
    if item_type == 'bud':
        loc = 'Office - Packed'
        loc_2 = 'Sales Floor'
    click(LOC_FILTER_3DOT)
    click(LOC_FILTER_OPTION)
    time.sleep(0.5)
    click(LOC_FILTER_LOGIC_BUTTON)
    time.sleep(1)
    click(LOC_FILTER_LOGIC_OR)
    enter(LOC_FILTER_INPUT_1, loc)
    enter(LOC_FILTER_INPUT_2, loc_2)
    click(LOC_FILTER_BUTTON)




In [11]:
row_caret_n = lambda x : f'/html/body/div[3]/div[2]/div[2]/div/div/div/div[3]/div/table/tbody/tr[{2*x-1}]/td[1]/a'
row_history_n = lambda x : f'/html/body/div[3]/div[2]/div[2]/div/div/div/div[3]/div/table/tbody/tr[{2*x}]/td[2]/div/div[1]/ul/li[5]/span[2]'
first_row_history = lambda x : f'/html/body/div[3]/div[2]/div[2]/div/div/div/div[3]/div/table/tbody/tr[{2*x}]/td[2]/div/div/div[5]/div/table/tbody/tr[1]/td[3]'
first_row_amount = lambda x : f'/html/body/div[3]/div[2]/div[2]/div/div/div/div[3]/div/table/tbody/tr[{2*x}]/td[2]/div/div/div[5]/div/table/tbody/tr[1]/td[1]'
date_toggle = lambda x : f'/html/body/div[3]/div[2]/div[2]/div/div/div/div[3]/div/table/tbody/tr[{2*x}]/td[2]/div/div/div[5]/div/table/thead/tr/th[3]/a[2]'

In [12]:
def get_history(row_num=1):
    click(row_caret_n(row_num))
    click(row_history_n(row_num))
    #wait for history table
    # create a for loop to try to find the history table every sec

    time.sleep(5)
    start_date = find(first_row_history(row_num)).text
    grams =find(first_row_amount(row_num)).text.split(' ')[1]

    click(date_toggle(row_num))
    click(date_toggle(row_num))
    time.sleep(1)
    end_date = find(first_row_history(row_num)).text
    click(date_toggle(row_num))
    click(row_caret_n(row_num))
    # print(f'{grams} - {start_date} : {end_date}')
    return {
                'start date': start_date,
                'end date': end_date,
                'amount' : grams,
            }


In [13]:
def get_sale_thru(num_hist):
    history = []
    row_num = 1
    while len(history) != num_hist:
        try:
            history.append(get_history(row_num))
            row_num+=1
        except Exception as e:
            print(e)
    return history


In [14]:
def create_historty_df(history):
    st_data = pd.DataFrame.from_records(history)
    st_data['start date'] = pd.to_datetime(st_data['start date'])
    st_data['end date'] = pd.to_datetime(st_data['end date'])
    st_data['amount'] = pd.to_numeric(st_data['amount'])
    st_data['duration'] = (st_data['end date'] - st_data['start date']).dt.days
    st_data['sale_thru'] = st_data['amount'] / st_data['duration']
    return st_data[st_data.duration != 0]

In [15]:
def make_history(num):
    data = get_sale_thru(num)
    df = create_historty_df(data)
    print(df.shape)
    return df

In [16]:
def send_data_to_airtable(data):
    pass

# MAIN FUNCTION

In [21]:
def run(item, item_type, hist_num):
    login()
    goto_inactive_pkg()
    time.sleep(0.5)
    filter_item(item)
    time.sleep(1.5)
    filter_loc(item_type)
    df = make_history(hist_num)
    df.to_csv(f'{item}_{item_type}_{hist_num}.csv')
    send_data_to_airtable(df)

In [18]:
PCK = 'Buds - Platinum Candy Kush'


In [22]:
run(PCK,'bud',7)

Message: Element <a class="k-icon k-i-expand" href="#"> is not clickable at point (75,289) because another element <div class="k-loading-image"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:279:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:160:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:119:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:208:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:97:31

Message: Element <a class="k-icon k-i-expand" href="#"> is not clickable at point (75,289) because another element <div class="k-loading-image"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8